<a href="https://colab.research.google.com/github/NastasiaMazur/Finance-Sentiment-Analysis/blob/main/distilroberta_base_FinanceIncauditor_sentiment_UPDATED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downstream Task: Sentiment Analysis

Model: **distilroberta-base**

Dataset: **FinanceInc/auditor_sentiment**

# 1. Activate GPU and Install Dependencies

In [1]:
#check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
# Install required libraries
!pip install transformers
!pip install datasets
!pip install bertviz transformers
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

In [3]:
# Conntext Google Drive to store data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 2. Preprocess data

In [4]:
# Load data
from datasets import load_dataset
finance_dataset = load_dataset("FinanceInc/auditor_sentiment")

# Create a smaller training dataset for faster training times
from datasets import DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(500)), # for training
    val=finance_dataset['train'].shuffle(seed=24).select(range(500, 600)), #  for validation
    test=finance_dataset['train'].shuffle(seed=24).select(range(600, 700)) # for testing
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3877 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/969 [00:00<?, ? examples/s]

In [5]:
small_finance_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 500
    })
    val: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 100
    })
})

In [6]:
small_finance_dataset['train'][:5]

{'sentence': ['FCC Chairman Kevin Martin said that fair play required extending the same deregulatory rules to the digital subscriber lines that telecom providers use for broadband networks .',
  'Metso Foundries Jyvaskyla Oy will discontinue production on this line by 30 September 2008 , the company said .',
  'Finnish business software group AffectoGenimap Oyj said its net profit halved to 1.2 mln euro ( $ 1.5 mln ) in the first nine months of 2006 from 2.2 mln euro ( $ 2.8 mln ) in the same period of 2005 .',
  'Finnish financial software developer Basware Oyj said today it will provide its invoice automation ( IA ) solution to an unnamed major retail company in the USA in a deal , worth more than EUR300 ,000 .',
  'According to the company , a decision in the issue will be made in the summer of 2010 , at the earliest , and in the summer of 2011 , at the latest .'],
 'label': [1, 1, 0, 2, 1]}

In [7]:
# Set Distilroberta-base tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
print(tokenizer)
print(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

RobertaTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
50265


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)
small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
small_tokenized_dataset['train'][0:3]


{'labels': tensor([1, 1, 0]),
 'input_ids': tensor([[    0,   597,  3376,  3356,  2363,  1896,    26,    14,  2105,   310,
           1552,  9148,     5,   276,   263,  4950, 25911,  1492,     7,     5,
           1778, 13707,  2301,    14,  9146,  4898,   304,    13, 11451,  4836,
            479,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1],
         [    0, 25869,  2527, 11911,  4458,   344,   219,   705,  4970,  4360,
            102, 17311,    40, 29649,  1780,   931,    15,    42,   516,    30,
            389,   772,  2266,  2156,     5,   138,    26,   479,     2,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1, 

In [10]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

# 3. Training the model

In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

In [12]:
# Define DistilROBERTA as our base model:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=3)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
num_epochs = 3
num_training_steps = 3 * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

Uncomment if you want to log in to your Hugging Face account:

In [15]:
from datasets import load_dataset, DatasetDict

small_finance_dataset = DatasetDict(
    train=finance_dataset['train'].shuffle(seed=24).select(range(300)),
    val=finance_dataset['train'].shuffle(seed=24).select(range(300, 400)),
    test=finance_dataset['train'].shuffle(seed=24).select(range(400, 500))
)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

small_tokenized_dataset = small_finance_dataset.map(tokenize_function, batched=True, batch_size=16)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
#from huggingface_hub import notebook_login

#notebook_login()

In [17]:
import numpy as np
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=3)

arguments = TrainingArguments(
    output_dir="store_the_checkpoints_distilroberta_3",                           #ADJUST FOLDER !!!
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    lr_scheduler_type='cosine',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    seed=224
)


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)


optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.887225,0.630000
2,No log,0.819598,0.630000
3,No log,0.699127,0.710000
4,No log,0.596780,0.730000
5,No log,0.551208,0.770000
6,No log,0.462987,0.800000
7,No log,0.454303,0.810000
8,No log,0.483320,0.790000
9,No log,0.469575,0.830000
10,No log,0.470563,0.830000


TrainOutput(global_step=190, training_loss=0.4929577074552837, metrics={'train_runtime': 78.4845, 'train_samples_per_second': 38.224, 'train_steps_per_second': 2.421, 'total_flos': 56375197595784.0, 'train_loss': 0.4929577074552837, 'epoch': 10.0})

In [19]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[-2.6007042e+00,  9.4050682e-01,  1.5110922e+00],
       [-1.5784671e+00, -1.1858029e+00,  2.2908831e+00],
       [-2.8523712e+00,  2.4274697e+00,  2.9208773e-01],
       [-2.8082771e+00,  1.9651399e+00,  6.8243015e-01],
       [-2.2737246e+00,  2.8922172e+00, -5.2952510e-01],
       [-2.5476813e+00,  2.2868376e+00,  1.0437093e-02],
       [-1.5955806e-01,  2.4143784e+00, -1.7159553e+00],
       [-1.4063337e+00,  7.3152155e-01,  5.9722823e-01],
       [-1.8376584e+00,  2.4830458e+00, -3.1141391e-01],
       [-1.4150523e+00,  3.0580277e+00, -1.3600430e+00],
       [-2.4190326e+00,  2.6115773e+00, -1.9850902e-01],
       [-2.4722831e+00,  2.9669192e+00, -4.4225886e-01],
       [-2.0438280e+00,  1.6212782e+00,  4.6809399e-01],
       [-2.8264759e+00,  1.1042749e+00,  1.4545047e+00],
       [-2.6134307e+00,  1.7166419e+00,  6.1064816e-01],
       [-2.0126467e+00,  3.1983490e+00, -8.3874786e-01],
       [-2.0255554e+00,  3.0531020e+00, -8.6442304e-01],
  

In [20]:
test_str = "The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_distilroberta_3/checkpoint-190") #ADJUST FOLDER !!!  500/600/700-folder320
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE", "NEUTRAL"][prediction])

NEUTRAL


In [21]:
results = trainer.predict(small_tokenized_dataset['test'])
print(results)

PredictionOutput(predictions=array([[-0.50996816,  2.701868  , -1.6591834 ],
       [-2.2986202 ,  2.935182  , -0.64685035],
       [-2.773372  ,  0.7217296 ,  1.603631  ],
       [-1.7936286 , -0.9811263 ,  2.255452  ],
       [-0.88719666,  3.2578428 , -1.7426585 ],
       [ 0.48317882,  2.0263393 , -1.7786802 ],
       [-1.9875886 ,  3.212105  , -1.0876145 ],
       [-0.7457497 , -1.5886935 ,  2.0696404 ],
       [-0.90629   , -1.5081625 ,  1.9900628 ],
       [-2.6954644 ,  1.7716857 ,  0.6786354 ],
       [-2.208523  ,  2.8752198 , -0.48442578],
       [ 0.99514735,  1.5729314 , -2.0182927 ],
       [-2.6208432 ,  1.0864108 ,  1.3090879 ],
       [-2.0481408 , -0.7110225 ,  2.3333397 ],
       [-1.8002713 ,  3.1553273 , -0.9719131 ],
       [-1.4314562 ,  0.5227799 ,  0.6848567 ],
       [-3.0120902 ,  1.7241937 ,  1.010135  ],
       [-2.7500968 ,  1.9722345 ,  0.7472779 ],
       [-2.447266  ,  1.9533299 ,  0.27756602],
       [ 1.7143917 , -0.05212933, -1.2499063 ],
       [-2.

In [22]:
small_finance_dataset['test'][5]

{'sentence': 'The company will also shut one paper machine in Finland and one in Austria , as well as two label paper machines in Finland for up to 10 months , Helsinki-based UPM said yesterday .',
 'label': 1}

In [23]:
fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("store_the_checkpoints_distilroberta_3/checkpoint-190") #ADJUST FOLDER !!!  500/600/700-folder320

model_inputs = tokenizer(small_tokenized_dataset['test']['sentence'], padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)

In [24]:
from torch.utils.tensorboard import SummaryWriter
import os
import re
import torch
import tensorflow as tf
import tensorboard as tb

In [25]:
len(outputs['hidden_states'])

7

In [26]:
import torch

path = "store_the_checkpoints_distilroberta_3/results_vis_distilroberta_2"
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != 0:
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_dataset['test']['sentence'][example],str(small_tokenized_dataset['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Sentence','Emotion'])

  layer+=1


 TensorFlow Embedding Projector [API](https://projector.tensorflow.org/).

# 4. Analyzing new data with the model

You can also use pipeline (uncomment a cell with huggingface above):

In [27]:
# Upload the model to the Hub
trainer.push_to_hub()

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-663252a4-6aeeef804eeabf1b37ea2218;69a8f20d-787b-41ff-b4c9-558c5a892e74)

Invalid username or password.

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="") # Add your saved mode in ""

sentiment_model(["The company's strong quarterly earnings report resulted in a surge in stock prices, reflecting investor confidence in its future prospects."])